# Работа с Excel

v 0.1

Датасет: себестоимостьА_в1.xlsx

In [3]:
import xlwings as xw
import numpy as np
import pandas as pd
from xlwings import constants 

## 1. Используя xlwings:

In [4]:
wb = xw.Book('себестоимостьА_в1.xlsx')
recipies = wb.sheets['Рецептура']

In [6]:
recipies.range('C7:C10').value

['Хлеб Венгерский', 'Хлеб "Барбари"', 'Багет парижский', 'Чиабатта']

1.1 На листе "Рецептура" для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [13]:
consumption = recipies.range('G7:O10').options(np.array).value
consumption = np.nan_to_num(consumption)
unit_prices = recipies.range('G14:O14').options(np.array).value
cost = consumption @ unit_prices

1.2 Результаты рассчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [14]:
# recipies.range('T7:T10').value = cost.reshape(-1, 1)
recipies.range('T6').value = 'Себестоимость'
recipies.range('T7').options(transpose=True).value = cost

1.3 Максимально приблизить форматирование столбца, добавленного в 1.2 к оформлению всей области.

In [ ]:
# cost = [round(i, 2) for i in cost]
recipies.range('T7:T10').options(transpose=True).value = cost
xw.Range('T7:T10').number_format = '0,00'
xw.Range('T7:T10').color = (255, 255, 167)
recipies.range('T4:T6').api.merge()
recipies.range('T6').value = 'Себестоимость'
xw.Range('T4:T6').color = (255, 192, 0)
xw.Range('T4:T6').autofit()

1.4 Заменить значения добавленные в 1.2 на формулы. 

In [16]:
xw.Range('U7').formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)' 

1.5 Выполнить 1.4 с помощью "протягиваемых" формул.

In [17]:
xw.Range('U7').formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)' 
from xlwings.constants import AutoFillType 
xw.Range('U7').api.AutoFill(xw.Range("U7:U10").api, 
                                AutoFillType.xlFillDefault)

True

1.6 Сдлеать так, чтобы решение 1.5 и 1.3 автоматически применилось ко всем аналогичным областям на листе "Рецептура"

## 2. Используя xlwings:

2.1 Собрать цены всех ресурсов, используемые на листе "Рецептура". Проверить, что нет расхождений между ценами совпадающих ресурсов

In [60]:
wb = xw.Book('себестоимостьА_в1.xlsx')
recipies = wb.sheets['Рецептура']
print(recipies.range('G14:O14').value)
print(recipies.range('G31:O31').value)
print(recipies.range('G52:Y52').value)
print(recipies.range('G73:Y73').value)

[7.0, 10.0, 184.0, 19.0, 7.0, 290.0, 105.0, 83.0, 260.0]
[7.0, 10.0, 10.0, 184.0, 19.0, 7.0, 260.0, 108.0, 0.0]
[7.0, 12.0, 12.0, 12.0, 12.0, 10.0, 184.0, 19.0, 7.0, 290.0, 40.0, 83.0, 260.0, 1.0, 18.0, 40.0, 83.0, 160.0, 0.0]
[7.0, 10.0, 184.0, 19.0, 7.0, 40.0, 150.0, 4.0, 260.0, 63.0, 110.0, 270.0, 83.0, 250.0, 280.0, 200.0, 80.0, 25.0, 93.0]


2.2 Вынести цены ресурсов в новую таблицу на новом листе "Цена ресурсов".

In [46]:
#wb.sheets.add(name="Цена ресурсов", after=wb.sheets[0].name)
res = wb.sheets['Цена ресурсов']
costs = recipies.range('G14:O14').value
res.range('A1').value = 'Пшеничный хлеб'
res.range('B1').value = costs

costs = recipies.range('G31:O31').value
res.range('A2').value = 'Ржаной хлеб'
res.range('B2').value = costs

costs = recipies.range('G52:Y52').value
res.range('A3').value = 'Хлеба специальной рецептуры'
res.range('B3').value = costs

costs = recipies.range('G73:Y73').value
res.range('A4').value = 'Слоеная выпечка'
res.range('B4').value = costs


2.3 Заменить фиксированные цены ресурсов на листе "Рецептура" на ссылки на лист "Цена ресурсов"

In [52]:
recipies.range('G14:O14').formula = "='Цена ресурсов'!B1"
recipies.range('G31:O31').formula = "='Цена ресурсов'!B2"
recipies.range('G52:Y52').formula = "='Цена ресурсов'!B3"
recipies.range('G73:Y73').formula = "='Цена ресурсов'!B4"

2.4 Сделать так, чтобы цены ресурсов на листе "Рецептура" подтягивались с листа "Цена ресурсов" по наименованию (подсказка: использовать ВПР, рекомендуется использовать именованные области)

In [57]:
res.range('A1').value

'Пшеничный хлеб'

## 3.

3.1 Создать нормализованную (не ниже 3й нормальной формы) базу данных sqlite для хранения информации из себестоимостьА_в1.xlsx .

Используя xlwings:

3.2 Информацию из себестоимостьА_в1.xlsx внести в БД на sqlite.

3.3 По базе данных sqlite сгенерирвать книгу эксель, по структуре аналогичную себестоимостьА_в1.xlsx

3.4 При помощи макроса на xlwings сделать в Excel кнопку, по которой все изменения на листе вносятся в БД, а цвет ячеек, в которых были найдены изменения, меняслся на зеленый.

3.5 Реализовать UDF, которая позволяет по имени ресурса вернуть количество видов в продукции в которых используется данный ресурс

3.6 Реализовать UDF, которая позволяет по имени ресурса вернуть массив наименований продуктов, в которых используется данный ресурс

3.7 Реазливать UDF 3.5 и 3.6 которые для получения ответа использую не книгу Excel а БД на sqlite.